# Nested Data and Explode Operations

This notebook demonstrates working with nested data structures and the explode operation in Polars.

In [ ]:
import os
import polars as pl
import json


## 1. Create and Load Nested Data


In [ ]:
datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
nested_data = [
    {'client_id': 'C0001', 'portfolios': [
        {'port_id': 'P1', 'holdings': ['A0001', 'A0002']},
        {'port_id': 'P2', 'holdings': ['A0003']}
    ]},
    {'client_id': 'C0002', 'portfolios': [
        {'port_id': 'P3', 'holdings': ['A0001', 'A0004']}
    ]}
]
with open(f'{datasets_dir}/nested.json', 'w') as f:
    json.dump(nested_data, f, indent=2)
df = pl.DataFrame(nested_data)
print('Original nested DataFrame:')
print(df)


## 2. Explode and Extract Struct Fields


In [ ]:
exploded = df.explode('portfolios')
extracted = exploded.with_columns([
    pl.col('portfolios').struct.field('port_id').alias('portfolio_id'),
    pl.col('portfolios').struct.field('holdings').alias('holdings')
])
print('Extracted struct fields:')
print(extracted)


## 3. Fully Explode Holdings


In [ ]:
fully_exploded = extracted.explode('holdings')
final = fully_exploded.drop('portfolios')
print('Fully exploded holdings:')
print(final)


## 4. Melt Operation (Wide to Long)


In [ ]:
wide_df = pl.DataFrame({
    'client_id': ['C0001', 'C0002', 'C0003'],
    'equity_val': [10000, 15000, 8000],
    'bond_val': [5000, 7000, 12000],
    'commodity_val': [2000, 0, 3000]
})
melted = wide_df.unpivot(index=['client_id'], on=['equity_val', 'bond_val', 'commodity_val'], variable_name='asset_class', value_name='value')
print('Melted to long format:')
print(melted)


## 5. Save Results


In [ ]:
final.write_csv(f'{datasets_dir}/exploded_holdings.csv')
melted.write_csv(f'{datasets_dir}/melted_assets.csv')
print('Saved exploded holdings and melted assets to CSV.')
